## setup

In [1]:
%pip install langchain langchain-openai langgraph python-dotenv

from dotenv import load_dotenv
import os

load_dotenv()

from dataclasses import dataclass
from langchain.agents import create_agent
from langchain.tools import tool, ToolRuntime
from langchain.chat_models import init_chat_model
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.structured_output import ToolStrategy


[notice] A new release of pip is available: 24.0 -> 26.0.1
[notice] To update, run: C:\Users\range\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/90.6 kB ? eta -:--:--
     ------------------------------------ --- 81.9/90.6 kB 4.5 MB/s eta 0:00:01
     -------------------------------------- 90.6/90.6 kB 737.1 kB/s eta 0:00:00
     ---------------------------------------- 0.0/43.1 kB ? eta -:--:--
     -------------------------------------- - 41.0/43.1 kB ? eta -:--:--
     -------------------------------------- 43.1/43.1 kB 420.9 kB/s eta 0:00:00
     ---------------------------------------- 0.0/57.7 kB ? eta -:--:--
     ----------------------------------- ---- 51.2/57.7 kB ? eta -:--:--
     ---------------------------------------- 57.7/57.7 kB 1.0 MB/s eta 0:00:00
     ---------------------------------------- 0.0/41.5 kB ? eta -:--:--
     ---------------------------------------- 41.5/41.5 kB 1.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/111.7 kB ? eta -:--:--
   ------------------------------------ --- 102.4/111.7 kB 5.8 MB/s eta 0:00:01
   ------------

## system promp

In [2]:
SYSTEM_PROMPT = """
You are an expert weather forecaster who always speaks in clever puns.

You have access to two tools:

- get_weather_for_location: use this to get weather for a specific city
- get_user_location: use this if the user asks about the weather "outside"
  or seems to mean their current location.

Always make sure you know the location before answering.
"""

## create tools

In [3]:
@dataclass
class Context:
    user_id: str


@tool
def get_weather_for_location(city: str) -> str:
    """Get weather for a given city"""
    return f"It's always sunny in {city}!"


@tool
def get_user_location(runtime: ToolRuntime[Context]) -> str:
    """Retrieve user location using user ID"""
    user_id = runtime.context.user_id
    return "Florida" if user_id == "1" else "San Francisco"

## configure model

In [ ]:
model = init_chat_model(
    "gpt-4o-mini",
    temperature=0.5,
    max_tokens=1000,
    timeout=10
)

## response format

In [ ]:
@dataclass
class ResponseFormat:
    punny_response: str
    weather_conditions: str | None = None

## add memory

In [ ]:
checkpointer = InMemorySaver()

## create and run agent

In [ ]:
agent = create_agent(
    model=model,
    system_prompt=SYSTEM_PROMPT,
    tools=[get_weather_for_location, get_user_location],
    context_schema=Context,
    response_format=ToolStrategy(ResponseFormat),
    checkpointer=checkpointer
)

In [ ]:
config = {"configurable": {"thread_id": "1"}}

## first ejecution

In [ ]:
response = agent.invoke(
    {"messages": [{"role": "user", "content": "What's the weather outside?"}]},
    config=config,
    context=Context(user_id="1")
)

print(response["structured_response"])

## second ejecution

In [ ]:
response = agent.invoke(
    {"messages": [{"role": "user", "content": "Thank you!"}]},
    config=config,
    context=Context(user_id="1")
)

print(response["structured_response"])